# **Long/Short Pairs Portfolio Algo**
### **Group：I Ching**

### **Import modules and databases**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Load the stockInfo, researchData and testData
stocksInfo = pd.read_excel('https://raw.githubusercontent.com/kenwkliu/ideas/master/colab/data/hkStocksQuotes.xlsx')
researchData = pd.read_csv('https://raw.githubusercontent.com/kenwkliu/ideas/master/colab/data/researchHKStocksAdjClosePx.csv', index_col=0)
testData = pd.read_csv('https://raw.githubusercontent.com/kenwkliu/ideas/master/colab/data/testHKStocksAdjClosePx.csv', index_col=0)

### **Define related functions**

In [32]:
# Functions
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

from datetime import datetime
import pandas_datareader.data as web


# Set the image size
CHART_SIZE_X, CHART_SIZE_Y = 12, 8
SMALL_CHART_SIZE_X, SMALL_CHART_SIZE_Y = 12, 8


def zscore(series):
    return (series - series.mean()) / np.std(series)


# Plot stock pair chart
def plotPair(df, stockA, stockB, sizeX, sizeY):
  ax1 = df[stockA].plot(label=stockA, legend=True, figsize = (sizeX, sizeY))
  ax1.set_ylim(df[stockA].min(), df[stockA].max())

  ax2 = df[stockB].plot(secondary_y=True, label=stockB, legend=True, figsize = (sizeX, sizeY))
  ax2.set_ylim(df[stockB].min(), df[stockB].max())

  plt.show()


# Plot all many Pairs at the same time
def plotManyPair(pairsDf):
  for index, row in pairsDf.iterrows():
      print('\n', index, ':', row['stockA'], 'vs', row['stockB'], '(', row['p-value'], ')')
      plotPair(researchData, row['stockA'], row['stockB'], SMALL_CHART_SIZE_X, SMALL_CHART_SIZE_Y)
    

# Filter the correlated stock pairs with the THRESHOLD
def getCorrelatedPairs(stocksCorr, THRESHOLD=-100):
  # filter the pairs with correlation values above the THRESHOLD
  highCorr = stocksCorr[((stocksCorr >= THRESHOLD) & (stocksCorr < 1))]
  highCorr = highCorr.unstack().sort_values(ascending=False).drop_duplicates()
  highCorr.dropna(inplace=True)
  highCorrDf = highCorr.to_frame().reset_index()
  highCorrDf.rename(columns = {'level_0':'stockA', 'level_1':'stockB', 0:'corr'}, inplace=True)

  # looks up the sectors for the stocksA and stockB
  cols = ['stockA', 'stockB', 'corr', 'sector_A', 'sector_B']
  pairsDf = highCorrDf.merge(stocksFilteredInfo[['shortName', 'sector']], how='left', left_on='stockA', right_on='shortName').merge(stocksFilteredInfo[['shortName', 'sector']], how='left', left_on='stockB', right_on='shortName', suffixes=('_A', '_B'))[cols]
  pairsDf['sameSector'] = (pairsDf['sector_A'] == pairsDf['sector_B'])
  
  return pairsDf

### back test related functions

# Determind the signal and dollarValue in the test data
# signal == -1: Long stockA Short stockB
# signal == 1: Short stockA Long stockB
# signal == 0: flat position
def backTest(testData, stockA, stockB, window1, window2):
  cols = [stockA, stockB]
  backTest_df = testData[cols].copy()
  backTest_df.dropna(inplace = True)

  # Set the tradingParams
  dollarValue = 20000

  # Calculate the Price ratio in backTest_df
  backTest_df['pxRatio'] = backTest_df[stockA] / backTest_df[stockB]
  backTest_df['ma1'] = backTest_df['pxRatio'].rolling(window=window1,center=False).mean()
  backTest_df['ma2'] = backTest_df['pxRatio'].rolling(window=window2,center=False).mean()
  backTest_df['std'] = backTest_df['pxRatio'].rolling(window=window2,center=False).std()
  backTest_df['zscore'] = (backTest_df['ma1'] - backTest_df['ma2'])/backTest_df['std']
  backTest_df['dollarValue'] = dollarValue

  # initialize the signal to 0
  backTest_df['signal'] = 0
  signal = 0

  backTest_df.dropna(inplace = True)
  
  # Determine the signal in each row of the backTest_df
  for index, row in backTest_df.iterrows():
    zscore = row['zscore']
    # mark signal = 1 if zscore > 1 (Diverge outside the upper band)
    if zscore > 1:
      signal = 1

    # mark signal = -1 if zscore < -1 (Diverge outside the lower band)
    elif zscore < -1:
      signal = -1

    else:
      # continue to mark signal = 1 if previous signal == 1 and zscore > 0.75 (Trade entered but not converge back yet)
      if signal == 1 and zscore > 0.75:
        signal = 1

      # continue to mark signal = -1 if previous signal == -1 and < -0.75 (Trade entered but not converge back yet)
      elif signal == -1 and zscore < -0.75:
        signal = -1

      else:
        signal = 0

    backTest_df.loc[index, 'signal'] = signal

  return backTest_df


# determine pSignal and nSignal for up/down markers in plot
# pSignal and nSignal is for displaying the up/down markers in plotting chart only, they're not required for backtest calculation
def addSignalMarker(backTest_df):
  backTest_df['pSignal'] = np.where(backTest_df['signal'] == 1, backTest_df['pxRatio'], np.nan)
  backTest_df['nSignal'] = np.where(backTest_df['signal'] == -1, backTest_df['pxRatio'], np.nan)

  return backTest_df


# Combine the research and backtest for a Portfolio of Pairs
def researchAndBackTestPortfolio(pairsDf, researchData, testData, window1, window2, printOutput=True):
  pairsPortfolioBackTest = []

  for index, row in pairsDf.iterrows():
    stockA, stockB = row['stockA'], row['stockB']
    if printOutput: print(stockA, 'vs', stockB)
    pairsPortfolioBackTest.append(backTest(testData, stockA, stockB, window1 ,window2)[[stockA, stockB, 'signal', 'dollarValue']])

  return pairsPortfolioBackTest



# Calculate the PnL of the Pair portfolio
def calcPortfolio(pairsBackTest):
    portfolio = []
    portfolioPnl = 0

    for backtestDf in pairsBackTest:
        pnl, pnlDf = calcPnl(backtestDf)

        stockA = backtestDf.columns[0]
        stockB = backtestDf.columns[1]
        print(stockA, 'vs', stockB, '---> $', pnl)

        portfolio.append([stockA, stockB, pnl])
        portfolioPnl += pnl

    print("=================================================================")
    print('PortfolioPnl: $', portfolioPnl)
    portfolioDf = pd.DataFrame(portfolio, columns=['stockA', 'stocksB', 'Pnl'])
    return portfolioPnl, portfolioDf


# Calculate the PnL of one Pair
def calcPnl(backTest_df):
    pnl_df = backTest_df.copy()

    pnl_df[['longPos', 'shortPos', 'longValue', 'shortValue', 'longPnl', 'shortPnl', 'pnl', 'totalPnl']] = 0
    signal, longPos, shortPos, longValue, shortValue, longPnl, shortPnl, pnl, accumPnl, totalPnl = 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0

    for index, row in pnl_df.iterrows():
        currentSignal = row['signal']

        if currentSignal != signal:
            if currentSignal == 0:
                longValue = 0
                shortValue = 0

            else:
                longValue = row['dollarValue']
                shortValue = row['dollarValue']

            longPos = getLongPos(currentSignal, longValue, row)
            shortPos = getShortPos(currentSignal, shortValue, row)

            longPnl = 0
            shortPnl = 0

            pnl = 0
            accumPnl = totalPnl

        #Store current row value
        pnl_df.loc[index, 'longPos'] = longPos
        pnl_df.loc[index, 'shortPos'] = shortPos

        curentLongValue = getLongValue(currentSignal, longPos, row)
        pnl_df.loc[index, 'longValue'] = curentLongValue

        curentShortValue = getShortValue(currentSignal, shortPos, row)
        pnl_df.loc[index, 'shortValue'] = curentShortValue

        currentLongPnl = curentLongValue - longValue
        pnl_df.loc[index, 'longPnl'] = currentLongPnl

        currentShortPnl = shortValue - curentShortValue
        pnl_df.loc[index, 'shortPnl'] = currentShortPnl

        currentPnl = currentLongPnl + currentShortPnl
        pnl_df.loc[index, 'pnl'] = currentPnl

        totalPnl = accumPnl + currentPnl
        pnl_df.loc[index, 'totalPnl'] = totalPnl

        signal = currentSignal

    finalPnl = pnl_df['totalPnl'].iloc[-1]
    return finalPnl, pnl_df


def getLongPos(signal, longValue, row):
    if longValue == 0 or signal == 0 : return 0
    if signal == -1: return longValue / row[0]
    if signal == 1: return longValue / row[1]


def getShortPos(signal, shortValue, row):
    if shortValue == 0 or signal == 0 : return 0
    if signal == -1: return shortValue / row[1]
    if signal == 1: return shortValue / row[0]


def getLongValue(signal, longPos, row):
    if longPos == 0 or signal == 0 : return 0
    if signal == -1: return longPos * row[0]
    if signal == 1: return longPos * row[1]


def getShortValue(signal, shortPos, row):
    if shortPos == 0 or signal == 0 : return 0
    if signal == -1: return shortPos * row[1]
    if signal == 1: return shortPos * row[0]

### **Select liquid equities**

In [9]:
# Select only liquid equity names for the trading purpose
# thus add a new colume 'turnover' 
stocksInfo['turnover'] = round(stocksInfo['regularMarketPreviousClose'] * stocksInfo['averageDailyVolume10Day'])

# Select only EQUITY and turnover is bigger than certain numbers
MIN_TURNOVER = 100000000  # 100 millions
SELECTED_COLUMNS = ['code', 'shortName', 'industry', 'sector', 'turnover']

stocksFilteredInfo = stocksInfo[(stocksInfo.turnover > MIN_TURNOVER) & (stocksInfo.industry != 'ETF')].reset_index()
stocksFilteredInfo.sort_values(by=['turnover'], ascending=False)[SELECTED_COLUMNS]

# The details of the 173 stocks listed by stocksFilteredInfo are in the researchData

,code,shortName,industry,sector,turnover
1,0700.HK,TENCENT,Information Technology,Online and Mobile Games,1.094491e+10
0,9988.HK,BABA-SW,Information Technology,E-commerce and Internet,6.289126e+09
27,0981.HK,SMIC,Information Technology,Semi-conductor,5.881712e+09
3,3690.HK,MEITUAN-W,Information Technology,E-commerce and Internet,4.727324e+09
8,0388.HK,HKEX,Financial,Other financial services,2.732908e+09
...,...,...,...,...,...
140,0136.HK,HENGTEN NET,Energy,Photovoltaic Solar,1.084515e+08
91,0880.HK,SJM HOLDINGS,Betting,Macau Gaming,1.077015e+08
158,3606.HK,FUYAO GLASS,Automobile production and distribution,Cars parts and maintenance,1.056159e+08
152,3933.HK,UNITED LAB,Medicine,Pharmaceutical,1.038379e+08


### **Look for stock pairs**

In [29]:
# Run the price correlation for all stock Pair combinations in research period
stocksCorr = researchData.corr()

In [30]:
pairsDf = getCorrelatedPairs(stocksCorr, -1)

pairsDf

,stockA,stockB,corr,sector_A,sector_B,sameSector
0,XINYI SOLAR,FLAT GLASS,0.986801,Photovoltaic Solar,Glass strands,False
1,MEITUAN-W,MEIDONG AUTO,0.981320,E-commerce and Internet,Auto sales,False
2,MAN WAH HLDGS,ZIJIN MINING,0.976942,Housewares,Precious metals,False
3,TENCENT,ZHONGSHENG HLDG,0.973716,Online and Mobile Games,Auto sales,False
4,XINYI GLASS,XINYI SOLAR,0.973291,Glass strands,Photovoltaic Solar,False
...,...,...,...,...,...,...
14873,CHINA TOWER,MEIDONG AUTO,-0.898294,Telecommunications Equipment,Auto sales,False
14874,MEIDONG AUTO,HEC PHARM,-0.902247,Auto sales,Pharmaceutical,False
14875,HEC PHARM,EVERSUNSHINE LS,-0.902860,Pharmaceutical,Property Management and Agent,False
14876,CHINA TOWER,KINGDEE INT'L,-0.914928,Telecommunications Equipment,Technology and Software,False


### **Backtesting**

In [31]:
# Selects the Pairs with same sectors into the portfolio
selectedPairsDf = pairsDf
print("Total pairs with same sectors for backtesting:", len(selectedPairsDf))
print('----------------------------------------------------------')

# Research the trading params and back test the selected Pairs in test period
pairsPortfolioBackTest = researchAndBackTestPortfolio(selectedPairsDf, researchData, testData, 5, 2, printOutput=False)

# Calcuate the PnL of the Pairs portfolio
pnl, pnlDf = calcPortfolio(pairsPortfolioBackTest)
pnlDf

Total pairs with same sectors for backtesting: 14878
----------------------------------------------------------
XINYI SOLAR vs FLAT GLASS ---> $ 4595.430174859521
MEITUAN-W vs MEIDONG AUTO ---> $ 10768.84551340024
MAN WAH HLDGS vs ZIJIN MINING ---> $ 3407.7160602555887
TENCENT vs ZHONGSHENG HLDG ---> $ 18890.92053538325
XINYI GLASS vs XINYI SOLAR ---> $ 15233.809131685517
MEITUAN-W vs ZHONGSHENG HLDG ---> $ 14847.75303106059
MAN WAH HLDGS vs XIAOMI-W ---> $ 1157.0610704206083
CG SERVICES vs YIHAI INTL ---> $ 7861.869176692962
MEITUAN-W vs TECHTRONIC IND ---> $ 13000.498782552308
XINYI SOLAR vs GREATWALL MOTOR ---> $ 3852.2853644108145
XINYI SOLAR vs WUXI BIO ---> $ 8760.278650015298
WUXI APPTEC vs PHARMARON ---> $ 5143.672368442039
MENGNIU DAIRY vs ZIJIN MINING ---> $ 11467.397846338496
MENGNIU DAIRY vs MAN WAH HLDGS ---> $ 9074.054546114265
MAN WAH HLDGS vs WUXI BIO ---> $ 6010.179537165743
XINYI GLASS vs XIAOMI-W ---> $ 3943.43465290753
XINYI GLASS vs FLAT GLASS ---> $ 18062.88126498

CHINA YOUZAN vs MEIDONG AUTO ---> $ 29181.4357971785
HENGTEN NET vs TECHTRONIC IND ---> $ 71203.0432811727
EB SECURITIES vs CITIC SEC ---> $ 7509.94565656437
EVERSUNSHINE LS vs MICROPORT ---> $ 3518.0184404004394
ZOOMLION vs HUABAO INTL ---> $ 32120.08510181191
BYD COMPANY vs XIAOMI-W ---> $ 7704.46352622658
FUYAO GLASS vs CM BANK ---> $ 6202.759161498645
MEITUAN-W vs BYD ELECTRONIC ---> $ 9020.47773993681
EVERSUNSHINE LS vs YIHAI INTL ---> $ 8636.544908992884
PHARMARON vs MAN WAH HLDGS ---> $ 8248.858391772916
WYNN MACAU vs CNOOC ---> $ 16546.018161873693
WUXI APPTEC vs MAN WAH HLDGS ---> $ 9285.618839177714
EVERG HEALTH vs HKEX ---> $ 26834.689686935402
XIAOMI-W vs FLAT GLASS ---> $ 17321.321431153865
XINYI GLASS vs SHENZHOU INTL ---> $ 8527.459107777322
FUYAO GLASS vs CHINASOFT INT'L ---> $ 12027.146211631672
TENCENT vs CHINA LIT ---> $ 14571.291949632592
FUYAO GLASS vs BYD COMPANY ---> $ 3730.4651083459175
HKEX vs MENGNIU DAIRY ---> $ 12650.996492149901
MEIDONG AUTO vs HKEX ---> $ 

WUXI BIO vs ANTA SPORTS ---> $ 5670.595248958838
MENGNIU DAIRY vs FUYAO GLASS ---> $ 13559.415851638256
KINGSOFT vs MEITUAN-W ---> $ 11097.878327049639
ZHONGSHENG HLDG vs MAN WAH HLDGS ---> $ 5997.2515366657535
MENGNIU DAIRY vs FLAT GLASS ---> $ 8532.175537393068
ABC vs ICBC ---> $ 3374.555893037621
HENGTEN NET vs HKEX ---> $ 50894.28749198973
CHINA LIT vs TECHTRONIC IND ---> $ 17550.153420358653
XINYI SOLAR vs GANFENGLITHIUM ---> $ 9864.463966833224
XINYI SOLAR vs WUXI APPTEC ---> $ 14665.407014698607
KINGSOFT vs KINGDEE INT'L ---> $ 7913.794056661049
CHINA RES BEER vs ZOOMLION ---> $ 11793.004914962992
TIANNENG POWER vs HKEX ---> $ 8180.006272166644
WUXI APPTEC vs MEIDONG AUTO ---> $ 11949.342529233512
CITIC SEC vs CGS ---> $ 7394.488232806689
TSINGTAO BREW vs ZOOMLION ---> $ 14236.459679516924
CPIC vs PING AN ---> $ 4966.857356509321
AIA vs LENOVO GROUP ---> $ 4184.8014399114
EB SECURITIES vs ZA ONLINE ---> $ 10052.114087889466
HKTV vs FLAT GLASS ---> $ 17007.562985207413
YIHAI INTL

BYD ELECTRONIC vs CG SERVICES ---> $ 9542.728442231019
GEELY AUTO vs BYD COMPANY ---> $ 10694.874310262974
CHINA LIT vs PHARMARON ---> $ 13628.30210028007
XINYI SOLAR vs HAIDILAO ---> $ 24181.54121165624
HKEX vs INNOVENT BIO ---> $ 18634.201461323926
HOPE EDU vs CANSINOBIO-B ---> $ 20246.37647687904
TENCENT vs MENGNIU DAIRY ---> $ 13779.643469238741
KINGSOFT vs MEILAN AIRPORT ---> $ 1304.8350826298229
MEIDONG AUTO vs KINGSOFT ---> $ 13631.429004993459
MEIDONG AUTO vs XIAOMI-W ---> $ 11269.72339746113
INNOVENT BIO vs EVERSUNSHINE LS ---> $ 7807.930919932391
HANG SENG BANK vs BANK OF CHINA ---> $ 1322.7263271265801
KINGDEE INT'L vs INNOVENT BIO ---> $ 9331.248036242896
TIANNENG POWER vs TSINGTAO BREW ---> $ 9382.345007963444
SUNAC vs BEIJING ENT ---> $ 10948.363786352358
CSC vs YIHAI INTL ---> $ 17403.483878672832
CHINASOFT INT'L vs XINYI SOLAR ---> $ 16021.640557913233
GANFENGLITHIUM vs XIAOMI-W ---> $ 2729.714652682749
MEIDONG AUTO vs LI NING ---> $ 11977.206416285113
HUABAO INTL vs FO

CHINA TAIPING vs ASM PACIFIC ---> $ 10484.98617599603
TIANNENG POWER vs SD GOLD ---> $ 7634.828715689022
AVICHINA vs XD INC ---> $ 29048.72498888803
WYNN MACAU vs CKH HOLDINGS ---> $ 7122.51815250485
CHINASOFT INT'L vs WUXI BIO ---> $ 252.07162158730716
MEIDONG AUTO vs XINYI GLASS ---> $ 10405.506988572248
WEIMOB INC vs MEILAN AIRPORT ---> $ 24863.643633542953
HK & CHINA GAS vs HSBC HOLDINGS ---> $ 4129.835547316645
HUABAO INTL vs MEITUAN-W ---> $ 20979.69322920344
LONGFOR GROUP vs MEIDONG AUTO ---> $ 10854.090648280915
CITIC BANK vs SUNAC ---> $ 4647.0153923284415
CHINA TAIPING vs PETROCHINA ---> $ 11822.25891078059
CICC vs EVERG HEALTH ---> $ 33076.94156116075
WUXI BIO vs ND PAPER ---> $ 11470.701019466655
KINGDEE INT'L vs MICROPORT ---> $ 16508.813532351283
ND PAPER vs BYD ELECTRONIC ---> $ 14685.84989379728
XIAOMI-W vs CHINA RES BEER ---> $ -600.5891225238265
VIVA BIOTECH vs MEILAN AIRPORT ---> $ 16758.884148856363
GANFENGLITHIUM vs MENGNIU DAIRY ---> $ 5701.761889425146
CHINA RES 

...

CHINA FEIHE vs WH GROUP ---> $ 10177.516838660958
CHINA GAS HOLD vs BROAD HOMES ---> $ 6585.529917821281
HK & CHINA GAS vs HUABAO INTL ---> $ 19510.017764607877
GANFENGLITHIUM vs HEC PHARM ---> $ 17298.47041513895
CHINA OVERSEAS vs SHENZHOU INTL ---> $ 17811.56702591106
CHINASOFT INT'L vs CHINA JINMAO ---> $ 9805.177031999328
MINSHENG BANK vs ANTA SPORTS ---> $ 9206.883351659013
CITIC BANK vs MEIDONG AUTO ---> $ 6964.14437602145
GUANGDONG INV vs SD GOLD ---> $ 12886.338114117047
AIR CHINA vs POLY PPT DEV ---> $ 13446.748818451091
PICC P&C vs MEILAN AIRPORT ---> $ 9299.017437199476
SHENZHOU INTL vs CHINA TOWER ---> $ 12523.382511124888
HAIDILAO vs ZTE ---> $ 16594.256602683836
TECHTRONIC IND vs CHINA JINMAO ---> $ 9174.578163961538
WHARF REIC vs AK MEDICAL ---> $ 25046.6900570541
ALI HEALTH vs PICC P&C ---> $ 8582.222309499186
AIR CHINA vs PA GOODDOCTOR ---> $ 14015.075291759362
MICROPORT vs HANG SENG BANK ---> $ 9554.685886848976
WUXI BIO vs FI2 CSOP HSI ---> $ 22320.770544379255
CHINA

BROAD HOMES vs ABC ---> $ 12501.123037467001
FI2 CSOP HSI vs ND PAPER ---> $ 11638.477502867561
CHINA MOBILE vs MICROPORT ---> $ 14488.45352638474
BABA-SW vs MINSHENG BANK ---> $ 6454.748010372608
CC NEW LIFE vs FUYAO GLASS ---> $ 8888.506473894973
GANFENGLITHIUM vs CHINA JINMAO ---> $ 11948.523474662336
BANK OF CHINA vs BROAD HOMES ---> $ 10817.422113661767
PICC P&C vs MICROPORT ---> $ 13612.602607765159
SUNART RETAIL vs HKTV ---> $ 10066.562095306988
HANSOH PHARMA vs HK & CHINA GAS ---> $ 8450.169862483308
HK & CHINA GAS vs HENGTEN NET ---> $ 79577.2110408833
CHINA OVERSEAS vs YIHAI INTL ---> $ 10897.325002567937
CHINA JINMAO vs MEITUAN-W ---> $ 2974.737227052172
CHINA MOBILE vs PHARMARON ---> $ 11892.497170575207
CITIC BANK vs CG SERVICES ---> $ 7982.415056074189
CANSINOBIO-B vs SHK PPT ---> $ 25589.960578174512
WEIMOB INC vs GUANGDONG INV ---> $ 29866.92091939791
SUNART RETAIL vs ZIJIN MINING ---> $ 16775.20596115529
CHINA RES BEER vs CHINA OVERSEAS ---> $ 6381.793548420777
AK MEDI

MAN WAH HLDGS vs HEC PHARM ---> $ 14849.574137831376
LENOVO GROUP vs SUNART RETAIL ---> $ 10422.887978880404
YIHAI INTL vs ICBC ---> $ 10421.542193043326
BUD APAC vs FI2 CSOP HSI ---> $ 6920.751478290113
PICC P&C vs YIHAI INTL ---> $ 9216.703737269454
SHENZHOU INTL vs MINSHENG BANK ---> $ 19131.680781306117
CHINA JINMAO vs BYD ELECTRONIC ---> $ 16029.96414964627
CHINA OVERSEAS vs WUXI APPTEC ---> $ 16049.671672985723
TSINGTAO BREW vs FI2 CSOP HSI ---> $ 4377.778178748475
LONGFOR GROUP vs MINSHENG BANK ---> $ 8598.694891439292
EVERSUNSHINE LS vs BANK OF CHINA ---> $ 3674.593495192974
GUANGDONG INV vs MEIDONG AUTO ---> $ 8254.726573065014
YIHAI INTL vs BANK OF CHINA ---> $ 10545.935849037745
CC NEW LIFE vs LI NING ---> $ 13351.808377793514
XD INC vs HK & CHINA GAS ---> $ 24807.83542833336
KINGDEE INT'L vs CHINA MOBILE ---> $ 15964.887629440376
GANFENGLITHIUM vs CHINA OVERSEAS ---> $ 10921.279325884825
MINSHENG BANK vs CHINA FEIHE ---> $ 9787.368080773333
CHINA OVERSEAS vs HUABAO INTL ---

WUXI APPTEC vs CHINA TOWER ---> $ 16751.349703874384
CHINA OVERSEAS vs HAIDILAO ---> $ 12409.287985043735
FLAT GLASS vs CHINA TOWER ---> $ 25805.918466784813
XIAOMI-W vs CHINA OVERSEAS ---> $ 12564.99932795699
SHENZHOU INTL vs SUNART RETAIL ---> $ 16132.49569063323
FI2 CSOP HSI vs GALAXY ENT ---> $ 5830.683654193195
CHINA TOWER vs LI NING ---> $ 12485.396903264042
KINGDEE INT'L vs CHINA OVERSEAS ---> $ 11810.219677776124
HKEX vs CHINA TOWER ---> $ 2680.9654578018453
CHINA TOWER vs HUABAO INTL ---> $ 18935.022441294976
MINSHENG BANK vs XINYI SOLAR ---> $ 20490.547628328623
CHINA TOWER vs BYD COMPANY ---> $ 13624.185160178691
YIHAI INTL vs HK & CHINA GAS ---> $ 11503.988714772528
CM BANK vs FI2 CSOP HSI ---> $ 12406.774187276322
FLAT GLASS vs CHINA OVERSEAS ---> $ 27638.819628719542
FOSUN PHARMA vs HEC PHARM ---> $ 31911.17607100012
FI2 CSOP HSI vs CMOC ---> $ 16049.546130971987
FUYAO GLASS vs SUNART RETAIL ---> $ 10113.447126525247
CHINA TOWER vs XINYI SOLAR ---> $ 22876.89488389399
PIC

,stockA,stocksB,Pnl
0,XINYI SOLAR,FLAT GLASS,4595.430175
1,MEITUAN-W,MEIDONG AUTO,10768.845513
2,MAN WAH HLDGS,ZIJIN MINING,3407.716060
3,TENCENT,ZHONGSHENG HLDG,18890.920535
4,XINYI GLASS,XINYI SOLAR,15233.809132
...,...,...,...
14873,CHINA TOWER,MEIDONG AUTO,6816.203319
14874,MEIDONG AUTO,HEC PHARM,17310.080062
14875,HEC PHARM,EVERSUNSHINE LS,4829.778485
14876,CHINA TOWER,KINGDEE INT'L,17118.132996


In [33]:
pnlDf.to_csv('pnl.csv', index=None)